In [1]:
# Import dependencies
import pandas as pd
from pyxlsb import open_workbook

import requests
import json

from config import eia_api_key

# Extract Data Into DataFrames

### Data 1: Irena.org Dataset on Electricity Installed Capacity (MW)

In [2]:
# Import data file
irena_eic_file = "data/IRENA_Stats_Tool.xlsb"

irena_eic_df = []
with open_workbook (irena_eic_file) as wb:
     with wb.get_sheet("Data") as sheet:
        for row in sheet.rows():
            irena_eic_df.append([item.v for item in row])

irena_eic_df = pd.DataFrame(irena_eic_df[8:], columns=irena_eic_df[5])
irena_eic_df.head()

,Region,Country,ISO Code,RE or Non-RE,Group Technology,Technology,Poducer Type,Year,Electricity Installed Capacity (MW),,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Africa,Algeria,DZA,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,2018.0,21308.0,None,...,None,None,None,None,None,None,None,None,None,None
1,Africa,Algeria,DZA,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,2019.0,23508.0,None,...,None,None,None,None,None,None,None,None,None,None
2,Africa,Algeria,DZA,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,2018.0,228.0,None,...,None,None,None,None,None,None,None,None,None,None
3,Africa,Algeria,DZA,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,2019.0,228.0,None,...,None,None,None,None,None,None,None,None,None,None
4,Africa,Algeria,DZA,Total Renewable,Solar energy,Concentrated solar power,On-grid,2018.0,25.0,None,...,None,None,None,None,None,None,None,None,None,None


### Data 2 (1): bp.com Dataset on Energy Consumption (Exajoules)

In [3]:
# Import data file
bp_pe_file = "data/bp-stats-review-2020-all-data.xlsx"

bp_pe = pd.read_excel(bp_pe_file, "Primary Energy Consumption", engine="openpyxl")

# Rename the columns with value from the second row
bp_pe_rename = bp_pe.rename(columns=bp_pe.iloc[1])
bp_pe_rename.head()

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2019,2008-18,2019,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2015.00000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,13.99365,13.936678,14.109482,14.349534,14.214058,-0.00944118,0.00640355,0.0243434,NaN,NaN
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.68830,7.785419,7.895341,7.833322,7.720742,-0.0143718,0.00905657,0.0132227,NaN,NaN


### Data 2 (2): bp.com Dataset on Renewables Consumption (Exajoules)

In [4]:
# Import data file
bp_rc_file = "data/bp-stats-review-2020-all-data.xlsx"

bp_rc = pd.read_excel(bp_rc_file, "Renewables Consumption - EJ", engine="openpyxl")

# Rename the columns with value from the second row
bp_rc_rename = bp_rc.rename(columns=bp_rc.iloc[1])
bp_rc_rename.head()

,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2019,2008-18,2019,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN
1,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.00000,...,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.437742,0.488314,0.492008,0.503491,0.523399,0.0395406,0.134528,0.0180629,NaN,NaN
4,Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00161,...,0.157067,0.170452,0.184147,0.216463,0.345292,0.595155,0.102784,0.0119163,NaN,NaN


 ### Data 3: wesr.unep.org Dataset on GHG Emission

In [5]:
# Import data file
unep_file = "data/wesr_unep_GHGemission.csv"

unep_df = pd.read_csv(unep_file, skiprows=3)
unep_df.head()

,INDICATOR NAME (unit),COUNTRY NAME,YEAR,VALUE
0,Greenhouse gas (GHG) emissions from agricultur...,China,1961,254109.75
1,Greenhouse gas (GHG) emissions from agricultur...,China,1962,256115.15
2,Greenhouse gas (GHG) emissions from agricultur...,China,1963,267398.78
3,Greenhouse gas (GHG) emissions from agricultur...,China,1964,288531.02
4,Greenhouse gas (GHG) emissions from agricultur...,China,1965,301524.32


 ### Data 4 (1): eia.gov API Dataset on CO2 Emission

#### China CO2 Emission

In [6]:
# Building query URL
china_co2_query_url = f"http://api.eia.gov/series/?api_key={eia_api_key}&series_id=INTL.4008-8-CHN-MMTCD.A"

In [7]:
# Get the info from JSON response
co2_response = requests.get(china_co2_query_url).json()

# Get the data to display for creating call format
print(json.dumps(co2_response, indent=4, sort_keys=False))

{
    "request": {
        "command": "series",
        "series_id": "INTL.4008-8-CHN-MMTCD.A"
    },
    "series": [
        {
            "series_id": "INTL.4008-8-CHN-MMTCD.A",
            "name": "CO2 emissions, China, Annual",
            "units": "million metric tonnes carbon dioxide",
            "f": "A",
            "copyright": "None",
            "source": "EIA, U.S. Energy Information Administration",
            "geography": "CHN",
            "start": "1980",
            "end": "2018",
            "updated": "2020-12-16T10:43:39-0500",
            "data": [
                [
                    "2018",
                    10633.716109407
                ],
                [
                    "2017",
                    10431.664013179
                ],
                [
                    "2016",
                    10433.034096167
                ],
                [
                    "2015",
                    10457.195827419
                ],
                [


#### Germany CO2 Emission

In [8]:
# Building query URL
germany_co2_query_url = f"http://api.eia.gov/series/?api_key={eia_api_key}&series_id=INTL.4008-8-DEU-MMTCD.A"

In [9]:
# Get the info from JSON response
co2_response_g = requests.get(germany_co2_query_url).json()

# Get the data to display for creating call format
print(json.dumps(co2_response_g, indent=4, sort_keys=False))

{
    "request": {
        "command": "series",
        "series_id": "INTL.4008-8-DEU-MMTCD.A"
    },
    "series": [
        {
            "series_id": "INTL.4008-8-DEU-MMTCD.A",
            "name": "CO2 emissions, Germany, Annual",
            "units": "million metric tonnes carbon dioxide",
            "f": "A",
            "copyright": "None",
            "source": "EIA, U.S. Energy Information Administration",
            "geography": "DEU",
            "start": "1980",
            "end": "2018",
            "updated": "2020-12-02T16:24:23-0500",
            "data": [
                [
                    "2018",
                    798.0084039
                ],
                [
                    "2017",
                    824.833881662
                ],
                [
                    "2016",
                    828.275513747
                ],
                [
                    "2015",
                    826.764602382
                ],
                [
        

#### USA CO2 Emission

In [10]:
# Building query URL
usa_co2_query_url = f"http://api.eia.gov/series/?api_key={eia_api_key}&series_id=INTL.4008-8-USA-MMTCD.A"

In [11]:
# Get the info from JSON response
co2_response_u = requests.get(usa_co2_query_url).json()

# Get the data to display for creating call format
print(json.dumps(co2_response_u, indent=4, sort_keys=False))

{
    "request": {
        "command": "series",
        "series_id": "INTL.4008-8-USA-MMTCD.A"
    },
    "series": [
        {
            "series_id": "INTL.4008-8-USA-MMTCD.A",
            "name": "CO2 emissions, United States, Annual",
            "units": "million metric tonnes carbon dioxide",
            "f": "A",
            "copyright": "None",
            "source": "EIA, U.S. Energy Information Administration",
            "geography": "USA",
            "start": "1949",
            "end": "2018",
            "updated": "2020-12-16T10:43:39-0500",
            "data": [
                [
                    "2018",
                    5284.406529
                ],
                [
                    "2017",
                    5130.576907
                ],
                [
                    "2016",
                    5170.51057
                ],
                [
                    "2015",
                    5262.842024
                ],
                [
         

### Data 4 (2): eia.gov API Dataset on Population

#### China

In [12]:
# Building query URL
query_url_c = f"http://api.eia.gov/series/?api_key={eia_api_key}&series_id=INTL.4702-33-CHN-THP.A"

# Get the info from JSON response
popu_response_c = requests.get(query_url_c).json()
# Get the data to display for creating call format
print(json.dumps(popu_response_c, indent=4, sort_keys=False))

{
    "request": {
        "command": "series",
        "series_id": "INTL.4702-33-CHN-THP.A"
    },
    "series": [
        {
            "series_id": "INTL.4702-33-CHN-THP.A",
            "name": "Population, China, Annual",
            "units": "people in thousands",
            "f": "A",
            "copyright": "None",
            "source": "EIA, U.S. Energy Information Administration",
            "geography": "CHN",
            "start": "1980",
            "end": "2019",
            "updated": "2020-01-13T12:35:29-0500",
            "data": [
                [
                    "2019",
                    1434512
                ],
                [
                    "2018",
                    1428392
                ],
                [
                    "2017",
                    1421755
                ],
                [
                    "2016",
                    1414805
                ],
                [
                    "2015",
                    140774

#### Germany

In [13]:
# Building query URL
query_url_g = f"http://api.eia.gov/series/?api_key={eia_api_key}&series_id=INTL.4702-33-DEU-THP.A"
    
# Get the info from JSON response
popu_response_g = requests.get(query_url_g).json()
# Get the data to display for creating call format
print(json.dumps(popu_response_g, indent=4, sort_keys=False))

{
    "request": {
        "command": "series",
        "series_id": "INTL.4702-33-DEU-THP.A"
    },
    "series": [
        {
            "series_id": "INTL.4702-33-DEU-THP.A",
            "name": "Population, Germany, Annual",
            "units": "people in thousands",
            "f": "A",
            "copyright": "None",
            "source": "EIA, U.S. Energy Information Administration",
            "geography": "DEU",
            "start": "1980",
            "end": "2019",
            "updated": "2020-12-04T10:41:41-0500",
            "data": [
                [
                    "2019",
                    83120.6
                ],
                [
                    "2018",
                    82934.16
                ],
                [
                    "2017",
                    82690.85
                ],
                [
                    "2016",
                    82391.95
                ],
                [
                    "2015",
                    8

#### USA

In [14]:
# Building query URL
query_url_u = f"http://api.eia.gov/series/?api_key={eia_api_key}&series_id=INTL.4702-33-USA-THP.A"
    
# Get the info from JSON response
popu_response_u = requests.get(query_url_u).json()
# Get the data to display for creating call format
print(json.dumps(popu_response_u, indent=4, sort_keys=False))

{
    "request": {
        "command": "series",
        "series_id": "INTL.4702-33-USA-THP.A"
    },
    "series": [
        {
            "series_id": "INTL.4702-33-USA-THP.A",
            "name": "Population, United States, Annual",
            "units": "people in thousands",
            "f": "A",
            "copyright": "None",
            "source": "EIA, U.S. Energy Information Administration",
            "geography": "USA",
            "start": "1980",
            "end": "2019",
            "updated": "2020-03-20T16:26:44-0400",
            "data": [
                [
                    "2019",
                    328219.2
                ],
                [
                    "2018",
                    326645.8
                ],
                [
                    "2017",
                    324917
                ],
                [
                    "2016",
                    322865.5
                ],
                [
                    "2015",
                

# Transform Data

### Data 1: Irena.org Dataset on Electricity Installed Capacity (MW)

In [15]:
# Get coloumns 
irena_eic_df.columns

Index([                             'Region',
                                   'Country',
                                  'ISO Code',
                              'RE or Non-RE',
                          'Group Technology',
                                'Technology',
                              'Poducer Type',
                                      'Year',
       'Electricity Installed Capacity (MW)',
                                          '',
                                          '',
                                          '',
                                          '',
                                        None,
                                        None,
                                        None,
                                        None,
                                        None,
                                        None,
                                        None,
                                        None],
      dtype='object')

In [16]:
# Extract wanted columns
eic_cols = ['Country','Year','RE or Non-RE','Group Technology','Technology','Poducer Type','Electricity Installed Capacity (MW)']
eic_countries = irena_eic_df[eic_cols].copy()
eic_countries.head()

,Country,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
0,Algeria,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,21308.0
1,Algeria,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,23508.0
2,Algeria,2018.0,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,228.0
3,Algeria,2019.0,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,228.0
4,Algeria,2018.0,Total Renewable,Solar energy,Concentrated solar power,On-grid,25.0


In [17]:
# Drop rows with empty values
eic_countries.dropna()

,Country,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
0,Algeria,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,21308.00
1,Algeria,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,23508.00
2,Algeria,2018.0,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,228.00
3,Algeria,2019.0,Total Renewable,Hydropower (excl. Pumped Storage),Renewable hydropower,On-grid,228.00
4,Algeria,2018.0,Total Renewable,Solar energy,Concentrated solar power,On-grid,25.00
...,...,...,...,...,...,...,...
2968,Venezuela,2019.0,Total Renewable,Solar energy,Off-grid Solar photovoltaic,Off-grid,3.00
2969,Venezuela,2018.0,Total Renewable,Solar energy,On-grid Solar photovoltaic,On-grid,1.00
2970,Venezuela,2019.0,Total Renewable,Solar energy,On-grid Solar photovoltaic,On-grid,2.32
2971,Venezuela,2018.0,Total Renewable,Wind energy,Onshore wind energy,Off-grid,71.28


#### Extracting data from my selected three countries: China, Germany, and United States

In [18]:
# Extracting China's data
# Reset index to "Country"
eic_china = eic_countries[eic_countries["Country"]=="China"]
eic_china.set_index("Country", inplace=True)
eic_china.head()

,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
Country,,,,,,
China,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,1126270.000
China,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,1168004.147
China,2018.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,44660.000
China,2019.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,48740.000
China,2018.0,Total Non-Renewable,Other non-renewable energy,Other non-renewable energy,On-grid,4575.000


In [19]:
# Extracting Germany's data
# Reset index to "Country"
eic_germany = eic_countries[eic_countries["Country"]=="Germany"]
eic_germany.set_index("Country", inplace=True)
eic_germany.head()

,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
Country,,,,,,
Germany,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,91415.0
Germany,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,91991.0
Germany,2018.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,10799.0
Germany,2019.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,10799.0
Germany,2018.0,Total Non-Renewable,Other non-renewable energy,Other non-renewable energy,On-grid,2332.0


In [20]:
# Extracting USA's data
# Reset index to "Country"
eic_usa = eic_countries[eic_countries["Country"]=="United States"]
eic_usa.set_index("Country", inplace=True)
eic_usa.head()

,Year,RE or Non-RE,Group Technology,Technology,Poducer Type,Electricity Installed Capacity (MW)
Country,,,,,,
United States,2018.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,750619.0
United States,2019.0,Total Non-Renewable,Fossil fuels,Fossil fuels,On-grid,743307.0
United States,2018.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,99629.0
United States,2019.0,Total Non-Renewable,Nuclear,Nuclear,On-grid,98266.3
United States,2018.0,Total Non-Renewable,Other non-renewable energy,Other non-renewable energy,On-grid,3292.5


### Data 2 (1): bp.com Dataset on Energy Consumption (Exajoules)

In [21]:
# Extract wanted columns
bp_ec_countries = bp_pe_rename.copy()
bp_ec_countries.head()

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2019,2008-18,2019,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2015.00000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,13.99365,13.936678,14.109482,14.349534,14.214058,-0.00944118,0.00640355,0.0243434,NaN,NaN
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.68830,7.785419,7.895341,7.833322,7.720742,-0.0143718,0.00905657,0.0132227,NaN,NaN


In [22]:
# Renaming the same name columns (2019) by creating specific name columns and duplicating the according data row
bp_ec_countries ["2019_share"] = bp_ec_countries.iloc [:,58]
bp_ec_countries ["2019_growth_rate_per_annum"] = bp_ec_countries.iloc [:,56]
bp_ec_countries ["2019_year"] = bp_ec_countries.iloc[:,55]
bp_ec_countries

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2018.0,2019.0,2019,2008-18,2019,NaN,NaN,2019_share,2019_growth_rate_per_annum,2019_year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN,Share,Growth rate per annum,NaN
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN,2019,2019,2019.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,14.349534,14.214058,-0.00944118,0.00640355,0.0243434,NaN,NaN,0.0243434,-0.00944118,14.214058
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.833322,7.720742,-0.0143718,0.00905657,0.0132227,NaN,NaN,0.0132227,-0.0143718,7.720742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# # Get a list of the column names and select the wanted columns 
ls = list(bp_ec_countries.columns)
columns=['Exajoules',
 1965.0,
 1966.0,
 1967.0,
 1968.0,
 1969.0,
 1970.0,
 1971.0,
 1972.0,
 1973.0,
 1974.0,
 1975.0,
 1976.0,
 1977.0,
 1978.0,
 1979.0,
 1980.0,
 1981.0,
 1982.0,
 1983.0,
 1984.0,
 1985.0,
 1986.0,
 1987.0,
 1988.0,
 1989.0,
 1990.0,
 1991.0,
 1992.0,
 1993.0,
 1994.0,
 1995.0,
 1996.0,
 1997.0,
 1998.0,
 1999.0,
 2000.0,
 2001.0,
 2002.0,
 2003.0,
 2004.0,
 2005.0,
 2006.0,
 2007.0,
 2008.0,
 2009.0,
 2010.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.0,
 2016.0,
 2017.0,
 2018.0,
 '2019_year', 
 '2019_growth_rate_per_annum',
 '2008-18',
 '2019_share']
bp_ec_countries = bp_ec_countries[columns]
bp_ec_countries

,Exajoules,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019_year,2019_growth_rate_per_annum,2008-18,2019_share
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2013.000000,2014.000000,2015.00000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,13.881791,14.031964,13.99365,13.936678,14.109482,14.349534,14.214058,-0.00944118,0.00640355,0.0243434
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.738309,7.697182,7.68830,7.785419,7.895341,7.833322,7.720742,-0.0143718,0.00905657,0.0132227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Rename the fist and 2008-2018 column
bp_ec_countries = bp_ec_countries.rename(columns={bp_ec_countries.columns[0]: "Country",
                                                bp_ec_countries.columns[57]: "2008-2018_growth_rate_per_annum"})
bp_ec_countries

,Country,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019_year,2019_growth_rate_per_annum,2008-2018_growth_rate_per_annum,2019_share
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share
1,Exajoules,1965.000000,1966.000000,1967.000000,1968.000000,1969.000000,1970.000000,1971.000000,1972.000000,1973.000000,...,2013.000000,2014.000000,2015.00000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,7.857612,...,13.881791,14.031964,13.99365,13.936678,14.109482,14.349534,14.214058,-0.00944118,0.00640355,0.0243434
4,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,1.812695,...,7.738309,7.697182,7.68830,7.785419,7.895341,7.833322,7.720742,-0.0143718,0.00905657,0.0132227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Extracting data from my selected three countries: China, Germany, and United States

In [25]:
# Extracting China's data
# Reset index to "Country"
# Rearrange display method
bp_ec_china = bp_ec_countries[bp_ec_countries["Country"]=="China"]
bp_ec_china.reset_index(inplace=True)
bp_ec_china = bp_ec_china.T
bp_ec_china.head()

# Reset index
bp_ec_china.reset_index(inplace=True)
bp_ec_china.head()

# Drop the first two rows and rename the columns
bp_ec_china = bp_ec_china.drop([0,1])
bp_ec_china = bp_ec_china.rename(columns={"index":"Year",
                                        0:"Exajoules (input-equivalent)"})
bp_ec_china.head()

# Find the lens of the data
len(bp_ec_china)

# Use the len number to multiply the "China" to create a column of country name
# Add the country column to the dataframe
# Reset index
china_name = ["China"] * len(bp_ec_china)
bp_ec_china["Country"] = china_name
bp_ec_china.reset_index(inplace=True)
bp_ec_china.head()

# Delete the index and set index to country
del bp_ec_china["index"]
bp_ec_china.set_index("Country", inplace=True)
bp_ec_china.head()

,Year,Exajoules (input-equivalent)
Country,,
China,1965,5.51604
China,1966,5.995
China,1967,5.39134
China,1968,5.44535
China,1969,6.62766


In [26]:
# Extracting Germany's data
# Reset index to "Country"
# Rearrange display method
bp_ec_germany = bp_ec_countries[bp_ec_countries["Country"]=="Germany"]
bp_ec_germany.reset_index(inplace=True)
bp_ec_germany = bp_ec_germany.T

# Reset index
bp_ec_germany.reset_index(inplace=True)

# Drop the first two rows and rename the columns
bp_ec_germany = bp_ec_germany.drop([0,1])
bp_ec_germany = bp_ec_germany.rename(columns={"index":"Year",
                    0:"Exajoules (input-equivalent)"})

# Use the len number to multiply the "Germany" to create a column of country name
# Add the country column to the dataframe
# Reset index
germany_name = ["Germany"] * len(bp_ec_germany)
bp_ec_germany["Country"] = germany_name
bp_ec_germany.reset_index(inplace=True)

# Delete the index and set index to country
del bp_ec_germany["index"]
bp_ec_germany.set_index("Country", inplace=True)
bp_ec_germany.head()

,Year,Exajoules (input-equivalent)
Country,,
Germany,1965,10.679
Germany,1966,10.7058
Germany,1967,10.6789
Germany,1968,11.4022
Germany,1969,12.3013


In [27]:
# Extracting USA's data
# Reset index to "Country"
# Rearrange display method
bp_ec_usa = bp_ec_countries[bp_ec_countries["Country"]=="US"]
bp_ec_usa.reset_index(inplace=True)
bp_ec_usa = bp_ec_usa.T

# Reset index
bp_ec_usa.reset_index(inplace=True)

# Drop the first two rows and rename the columns
bp_ec_usa = bp_ec_usa.drop([0,1])
bp_ec_usa = bp_ec_usa.rename(columns={"index":"Year",
                    0:"Exajoules (input-equivalent)"})

# Use the len number to multiply the "United States" to create a column of country name
# Add the country column to the dataframe
# Reset index
usa_name = ["United States"] * len(bp_ec_usa)
bp_ec_usa["Country"] = usa_name
bp_ec_usa.reset_index(inplace=True)

# Delete the index and set index to country
del bp_ec_usa["index"]
bp_ec_usa.set_index("Country", inplace=True)
bp_ec_usa.head()

,Year,Exajoules (input-equivalent)
Country,,
United States,1965,52.4338
United States,1966,55.383
United States,1967,57.3118
United States,1968,60.7991
United States,1969,64.0496


### Data 2 (2): bp.com Dataset on Renewables Consumption (Exajoules)

In [28]:
# Extract wanted columns
bp_rc_countries = bp_rc_rename.copy()
bp_rc_countries.head()

,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2019,2008-18,2019,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN
1,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.00000,...,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.437742,0.488314,0.492008,0.503491,0.523399,0.0395406,0.134528,0.0180629,NaN,NaN
4,Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00161,...,0.157067,0.170452,0.184147,0.216463,0.345292,0.595155,0.102784,0.0119163,NaN,NaN


In [29]:
# Renaming the same name columns (2019) by creating specific name columns and duplicating the according data row
bp_rc_countries ["2019_share"] = bp_rc_countries.iloc [:,58]
bp_rc_countries ["2019_growth_rate_per_annum"] = bp_rc_countries.iloc [:,56]
bp_rc_countries ["2019_year"] = bp_rc_countries.iloc[:,55]
bp_rc_countries

,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2018.0,2019.0,2019,2008-18,2019,NaN,NaN,2019_share,2019_growth_rate_per_annum,2019_year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Growth rate per annum,NaN,Share,NaN,NaN,Share,Growth rate per annum,NaN
1,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.00000,...,2018.000000,2019.000000,2019,2008-18,2019,NaN,NaN,2019,2019,2019.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.503491,0.523399,0.0395406,0.134528,0.0180629,NaN,NaN,0.0180629,0.0395406,0.523399
4,Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00161,...,0.216463,0.345292,0.595155,0.102784,0.0119163,NaN,NaN,0.0119163,0.595155,0.345292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,♦ Less than 0.05%.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,n/a not available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,"USSR includesCIS, Georgia, Ukraine and the Bal...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,"# Excludes Estonia, Latvia and Lithuania pri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# # Get a list of the column names and select the wanted columns 
rc_ls = list(bp_rc_countries.columns)
rc_columns=[
'Exajoules (input-equivalent)',
 1965.0,
 1966.0,
 1967.0,
 1968.0,
 1969.0,
 1970.0,
 1971.0,
 1972.0,
 1973.0,
 1974.0,
 1975.0,
 1976.0,
 1977.0,
 1978.0,
 1979.0,
 1980.0,
 1981.0,
 1982.0,
 1983.0,
 1984.0,
 1985.0,
 1986.0,
 1987.0,
 1988.0,
 1989.0,
 1990.0,
 1991.0,
 1992.0,
 1993.0,
 1994.0,
 1995.0,
 1996.0,
 1997.0,
 1998.0,
 1999.0,
 2000.0,
 2001.0,
 2002.0,
 2003.0,
 2004.0,
 2005.0,
 2006.0,
 2007.0,
 2008.0,
 2009.0,
 2010.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.0,
 2016.0,
 2017.0,
 2018.0,
'2019_year',
'2019_growth_rate_per_annum',
 '2008-18',
 '2019_share'
]
bp_rc_countries = bp_rc_countries[rc_columns]
bp_rc_countries

,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019_year,2019_growth_rate_per_annum,2008-18,2019_share
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share
1,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.00000,...,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.290927,0.303300,0.437742,0.488314,0.492008,0.503491,0.523399,0.0395406,0.134528,0.0180629
4,Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00161,...,0.111551,0.133273,0.157067,0.170452,0.184147,0.216463,0.345292,0.595155,0.102784,0.0119163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,♦ Less than 0.05%.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,n/a not available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,"USSR includesCIS, Georgia, Ukraine and the Bal...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,"# Excludes Estonia, Latvia and Lithuania pri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Rename the fist and 2008-2018 column
bp_rc_countries = bp_rc_countries.rename(columns={bp_rc_countries.columns[0]: "Country",
                                                bp_rc_countries.columns[57]: "2008-2018_growth_rate_per_annum"})
bp_rc_countries

,Country,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019_year,2019_growth_rate_per_annum,2008-2018_growth_rate_per_annum,2019_share
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growth rate per annum,NaN,Share
1,Exajoules (input-equivalent),1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.00000,...,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2019,2008-18,2019
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.290927,0.303300,0.437742,0.488314,0.492008,0.503491,0.523399,0.0395406,0.134528,0.0180629
4,Mexico,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00161,...,0.111551,0.133273,0.157067,0.170452,0.184147,0.216463,0.345292,0.595155,0.102784,0.0119163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,♦ Less than 0.05%.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,n/a not available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,"USSR includesCIS, Georgia, Ukraine and the Bal...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,"# Excludes Estonia, Latvia and Lithuania pri...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Extracting data from my selected three countries: China, Germany, and United States

In [32]:
# Extracting China's data
# Reset index to "Country"
# Rearrange display method
bp_rc_china = bp_rc_countries[bp_rc_countries["Country"]=="China"]
bp_rc_china.reset_index(inplace=True)
bp_rc_china = bp_rc_china.T
bp_rc_china.head()

,0
index,91
Country,China
1965.0,0
1966.0,0
1967.0,0


In [33]:
# Reset index
bp_rc_china.reset_index(inplace=True)
bp_rc_china.head()

,index,0
0,index,91
1,Country,China
2,1965,0
3,1966,0
4,1967,0


In [34]:
# Drop the first two rows and rename the columns
bp_rc_china = bp_rc_china.drop([0,1])
bp_rc_china = bp_rc_china.rename(columns={"index":"Year",
                                        0:"Exajoules (input-equivalent)"})
bp_rc_china.head()

,Year,Exajoules (input-equivalent)
2,1965,0
3,1966,0
4,1967,0
5,1968,0
6,1969,0


In [35]:
# Find the lens of the data
len(bp_rc_china)

58

In [36]:
# Use the len number to multiply the "China" to create a column of country name
# Add the country column to the dataframe
# Reset index
china_name = ["China"] * len(bp_rc_china)
bp_rc_china["Country"] = china_name
bp_rc_china.reset_index(inplace=True)
bp_rc_china.head()

,index,Year,Exajoules (input-equivalent),Country
0,2,1965,0,China
1,3,1966,0,China
2,4,1967,0,China
3,5,1968,0,China
4,6,1969,0,China


In [37]:
# Delete the index and set index to country
del bp_rc_china["index"]
bp_rc_china.set_index("Country", inplace=True)
bp_rc_china.head()

,Year,Exajoules (input-equivalent)
Country,,
China,1965,0
China,1966,0
China,1967,0
China,1968,0
China,1969,0


In [38]:
# Extracting Germany's data
# Reset index to "Country"
bp_rc_germany = bp_rc_countries[bp_rc_countries["Country"]=="Germany"]
bp_rc_germany.set_index("Country", inplace=True)
bp_rc_germany.head()

,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,...,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019_year,2019_growth_rate_per_annum,2008-2018_growth_rate_per_annum,2019_share
Country,,,,,,,,,,,,,,,,,,,,,
Germany,0.0,0.0,0.0,0.0,0.0,0.0092,0.01001,0.010187,0.010531,0.010762,...,1.306625,1.429576,1.659036,1.643934,1.880427,1.969729,2.117454,0.0749974,0.0933409,0.073075


In [39]:
# Extracting Germany's data
# Reset index to "Country"
# Rearrange display method
bp_rc_germany = bp_rc_countries[bp_rc_countries["Country"]=="Germany"]
bp_rc_germany.reset_index(inplace=True)
bp_rc_germany = bp_rc_germany.T

# Reset index
bp_rc_germany.reset_index(inplace=True)

# Drop the first two rows and rename the columns
bp_rc_germany = bp_rc_germany.drop([0,1])
bp_rc_germany = bp_rc_germany.rename(columns={"index":"Year",
                    0:"Exajoules (input-equivalent)"})

# Use the len number to multiply the "Germany" to create a column of country name
# Add the country column to the dataframe
# Reset index
germany_name = ["Germany"] * len(bp_rc_germany)
bp_rc_germany["Country"] = germany_name
bp_rc_germany.reset_index(inplace=True)

# Delete the index and set index to country
del bp_rc_germany["index"]
bp_rc_germany.set_index("Country", inplace=True)
bp_rc_germany.head()

,Year,Exajoules (input-equivalent)
Country,,
Germany,1965,0
Germany,1966,0
Germany,1967,0
Germany,1968,0
Germany,1969,0


In [40]:
# Extracting USA's data
# Reset index to "Country"
# Rearrange display method
bp_rc_usa = bp_rc_countries[bp_rc_countries["Country"]=="US"]
bp_rc_usa.reset_index(inplace=True)
bp_rc_usa = bp_rc_usa.T

# Reset index
bp_rc_usa.reset_index(inplace=True)

# Drop the first two rows and rename the columns
bp_rc_usa = bp_rc_usa.drop([0,1])
bp_rc_usa = bp_rc_usa.rename(columns={"index":"Year",
                    0:"Exajoules (input-equivalent)"})

# Use the len number to multiply the "United States" to create a column of country name
# Add the country column to the dataframe
# Reset index
usa_name = ["United States"] * len(bp_rc_usa)
bp_rc_usa["Country"] = usa_name
bp_rc_usa.reset_index(inplace=True)

# Delete the index and set index to country
del bp_rc_usa["index"]
bp_rc_usa.set_index("Country", inplace=True)
bp_rc_usa.head()

,Year,Exajoules (input-equivalent)
Country,,
United States,1965,0.133322
United States,1966,0.14062
United States,1967,0.140736
United States,1968,0.15546
United States,1969,0.162271


### Data 3: wesr.unep.org Dataset on GHG Emission

In [41]:
# Get the column names and rearrange them 
unep_df_countries = unep_df.copy()
col_list = list(unep_df_countries.columns)
new_col = ['COUNTRY NAME','INDICATOR NAME (unit)', 'YEAR', 'VALUE']
unep_df_countries = unep_df_countries[new_col]
unep_df_countries

,COUNTRY NAME,INDICATOR NAME (unit),YEAR,VALUE
0,China,Greenhouse gas (GHG) emissions from agricultur...,1961,254109.75
1,China,Greenhouse gas (GHG) emissions from agricultur...,1962,256115.15
2,China,Greenhouse gas (GHG) emissions from agricultur...,1963,267398.78
3,China,Greenhouse gas (GHG) emissions from agricultur...,1964,288531.02
4,China,Greenhouse gas (GHG) emissions from agricultur...,1965,301524.32
...,...,...,...,...
383,China,"Greenhouse gas (GHG) emissions, total (metric ...",2011,12234.29
384,China,"Greenhouse gas (GHG) emissions, total (metric ...",2012,12488.24
385,China,"Greenhouse gas (GHG) emissions, total (metric ...",2013,12982.92
386,China,"Greenhouse gas (GHG) emissions, total (metric ...",2014,13113.21


#### Extracting data from the selected year of 2030 and 2050

In [42]:
# Choose the goal year 2030
# Reset index to "COUNTRY NAME"
unep_df_2030 = unep_df_countries[unep_df_countries["YEAR"]== "2030"]
unep_df_2030.set_index("COUNTRY NAME", inplace=True)
unep_df_2030

,INDICATOR NAME (unit),YEAR,VALUE
COUNTRY NAME,,,
China,Greenhouse gas (GHG) emissions from agricultur...,2030,811210.50
United States of America,Greenhouse gas (GHG) emissions from agricultur...,2030,361695.80
Germany,Greenhouse gas (GHG) emissions from agricultur...,2030,57838.37


In [43]:
# Choose the goal year 2050
# Drop duplicate country
# Reset index to "COUNTRY NAME"
unep_df_2050 = unep_df_countries[unep_df_countries["YEAR"]== "2050"].copy()
unep_df_2050.drop_duplicates("COUNTRY NAME", inplace=True)
unep_df_2050.set_index("COUNTRY NAME", inplace=True)
unep_df_2050

,INDICATOR NAME (unit),YEAR,VALUE
COUNTRY NAME,,,
China,Greenhouse gas (GHG) emissions from agricultur...,2050,875477.40
United States of America,Greenhouse gas (GHG) emissions from agricultur...,2050,366640.75
Germany,Greenhouse gas (GHG) emissions from agricultur...,2050,55460.54


 ### Data 4: eia.gov API Dataset on CO2 Emission, Renewable Electricity Net Generation, & Population

#### China

In [44]:
# Locate the wanted data from the json format output
co2_china = co2_response["series"][0]["data"]
len(co2_china)

39

In [45]:
# Create a list of number for the range of the data
# Create keys to hold the year and emission values
# Loop through the list of data to append the needed info list
range_numbers = [*range(0, 39, 1)]

co2_china_year = []
co2_china_emission = []
country_name = []

for number in range_numbers:
    list_set = co2_response["series"][0]["data"][number]
    year = list_set[0]
    emission = list_set[1]
    name = "China"
    co2_china_year.append(year)
    co2_china_emission.append(emission)
    country_name.append(name)

In [46]:
# Create a data frame to hold the keys
co2_china_emission_dic = {
    "Country":country_name,
    "Year":co2_china_year,
    "Emission":co2_china_emission
}

In [47]:
# Create a data frame
# Change it to ascending order
# Reset index to "Country"
china_co2_info_df = pd.DataFrame(co2_china_emission_dic)
china_co2_info_df = china_co2_info_df.sort_values(by="Year", ascending=True)
china_co2_info_df.set_index("Country", inplace=True)
china_co2_info_df.head()

,Year,Emission
Country,,
China,1980,1595.966030
China,1981,1586.506160
China,1982,1670.253074
China,1983,1784.999566
China,1984,1947.119365


#### Germany

In [48]:
# Locate the wanted data from the json format output
co2_germany = co2_response_g["series"][0]["data"]

# Create a list of number for the range of the data
# Create keys to hold the year and emission values
# Loop through the list of data to append the needed info list
range_numbers = [*range(0, len(co2_germany), 1)]

co2_germany_year = []
co2_germany_emission = []
country_name_g = []

for number in range_numbers:
    list_set = co2_response_g["series"][0]["data"][number]
    year = list_set[0]
    emission = list_set[1]
    name = "Germany"
    co2_germany_year.append(year)
    co2_germany_emission.append(emission)
    country_name_g.append(name)

# Create a data frame to hold the keys
co2_germany_emission_dic = {
    "Country":country_name_g,
    "Year":co2_germany_year,
    "Emission":co2_germany_emission
}

# Create a data frame
# Change it to ascending order
# Reset index to "Country"
germany_co2_info_df = pd.DataFrame(co2_germany_emission_dic)
germany_co2_info_df = germany_co2_info_df.sort_values(by="Year", ascending=True)
germany_co2_info_df.set_index("Country", inplace=True)
germany_co2_info_df.head()

,Year,Emission
Country,,
Germany,1980,--
Germany,1981,--
Germany,1982,--
Germany,1983,--
Germany,1984,--


#### USA

In [49]:
# Locate the wanted data from the json format output
co2_usa = co2_response_u["series"][0]["data"]

# Create a list of number for the range of the data
# Create keys to hold the year and emission values
# Loop through the list of data to append the needed info list
range_numbers = [*range(0, len(co2_usa), 1)]

co2_usa_year = []
co2_usa_emission = []
country_name_u = []

for number in range_numbers:
    list_set = co2_response_u["series"][0]["data"][number]
    year = list_set[0]
    emission = list_set[1]
    name = "USA"
    co2_usa_year.append(year)
    co2_usa_emission.append(emission)
    country_name_u.append(name)

# Create a data frame to hold the keys
co2_usa_emission_dic = {
    "Country":country_name_u,
    "Year":co2_usa_year,
    "Emission":co2_usa_emission
}

# Create a data frame
# Change it to ascending order
# Reset index to "Country"
usa_co2_info_df = pd.DataFrame(co2_usa_emission_dic)
usa_co2_info_df = usa_co2_info_df.sort_values(by="Year", ascending=True)
usa_co2_info_df.set_index("Country", inplace=True)
usa_co2_info_df.head()

,Year,Emission
Country,,
USA,1949,2206.690829
USA,1950,2382.046176
USA,1951,2526.687327
USA,1952,2473.373964
USA,1953,2536.892888


### Data 4 (2): eia.gov API Dataset on Population

In [ ]:
# Locate the wanted data from the json format output
popu_china = popu_response_c["series"][0]["data"]

# Create a list of number for the range of the data
# Create keys to hold the year and emission values
# Loop through the list of data to append the needed info list
range_numbers = [*range(0, len(popu_china), 1)]

popu_year_c = []
popu_c = []
country_name_c = []

for number in range_numbers:
    list_set = popu_response_c["series"][0]["data"][number]
    year = list_set[0]
    population = list_set[1]
    name = "China"
    popu_year_c.append(year)
    popu_c.append(popu_c)
    country_name_c.append(name)
    
# Create a data frame to hold the keys
popu_china_dic = {
    "Country":country_name_c,
    "Year":popu_year_c,
    "Population":popu_c
}

# Create a data frame
# Change it to ascending order
# Reset index to "Country"
popu_china_df = pd.DataFrame(popu_china_dic)
popu_china_df = popu_china_df.sort_values(by="Year", ascending=True)
popu_china_df.set_index("Country", inplace=True)
popu_china_df.head()